In [ ]:
%matplotlib inline

import csv
import cv2
import math
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle

from sklearn.model_selection import train_test_split

def load_sample_list(path, test_size=0.2, zeros_rate=0.3):
    
    samples = []
    with open(path+'/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            for i in range(3):
                line[i] = path + '/' + line[i].strip()
            samples.append(line)
    
    samples.pop(0)
    
    shuffle(samples)

    o0 = []
    o1 = []
    
    for sample in samples:
        if float(sample[3]):
            o1.append(sample)
        else:
            o0.append(sample)

    samples = o0[:math.floor(len(o1)*zeros_rate)] + o1
    
    train_samples, validation_samples = train_test_split(samples, test_size=test_size) 
    
    return(train_samples, validation_samples)

def generate_samples(samples, batch_size=32, cameras=('l','c','r'), flip=True):
    
    expanded = []
    steering_adj = 0.2
    
    if flip:
        flips = [False, True]
    else:
        flips = [False]
    
    for sample in samples:
        for camera in cameras:
            for flip in flips:
                expanded.append((sample) + [camera, flip,])     
    
    num_samples = len(expanded)
    
    while 1:
        shuffle(expanded)  
        for offset in range(0, num_samples, batch_size):
            x = []
            y = []            
            for batch_sample in expanded[offset:offset+batch_size]:
                center,left,right,steering,throttle,brake,speed,camera,flip = batch_sample
                steering, throttle, brake, speed = float(steering), float(throttle), float(brake), float(speed)
                if camera == 'c':
                    filename = center
                if camera == 'l':
                    steering += steering_adj
                    filename = left
                if camera == 'r':
                    steering -= steering_adj
                    filename = right
                image = plt.imread(filename)
                if flip:        
                    image = np.fliplr(image) 
                    steering = -steering
                x.append(image)
                y.append(steering)
                
            yield(np.array(x),np.array(y))

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, Dropout

train, valid = load_sample_list('data')

# compile and train the model using the generator function
train_generator = generate_samples(train, batch_size=32)
validation_generator = generate_samples(valid, batch_size=32)

rate = .5
    
model = Sequential()
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: x/127.5 - 1.))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='elu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='elu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='elu'))
model.add(Convolution2D(64,3,3,activation='elu'))
model.add(Convolution2D(64,3,3,activation='elu'))
model.add(Flatten())
model.add(Dropout(rate))
model.add(Dense(1154))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
expansion = 6
epochs = 2
model.fit_generator(train_generator, samples_per_epoch= \
            len(train)*expansion, validation_data=validation_generator, \
            nb_val_samples=len(valid)*expansion, nb_epoch=epochs)

model.save('model.h5')


Using TensorFlow backend.


Epoch 1/2
 7680/22926 [=========>....................] - ETA: 23s - loss: 0.1136